In [2]:
import sys
import os

root_dir = os.path.split(os.getcwd())[0]

sys.path.append(root_dir)
from utils.helper_metastore import *
from utils.configurations.config import Config

In [3]:
import warnings
warnings.filterwarnings('ignore', 'absl')

%load_ext tensorboard

## Model Training

Here comes the most important part of our pipeline (model training). The whole pipeline that we are trying to build on are jointly called as an Continous training component in the MLOps workflow. The main moto over here is to keep our model upto date in the production and the counter back the data drift, model drift and training-serving skew.

![mlops pipeline](image/MLOps_pipeline.svg)

In this notebook, we cover the model training process as part of a machine learning pipeline, including how it is automated in a TFX pipeline. We also include some details of distribution strategies available in TensorFlow and how to tune hyperparameters in a pipeline. This chapter is more specific to TFX pipelines than most of the
others because we don’t cover training as a standalone process.

One very important feature of training a model in a TFX pipeline is that the data pre-processing step which ar saved along with the trained model weights. This is very useful once our model is deployed to production because it means that the preprocessing steps will always produce the features the models expecting. Without this feature, it would be possible to update the data pre‐processing steps without updating the model, and then the model would fail in production. So why TFX eports the proprocessing steps and the model as one graph, this potentialy eliminates the source of error.

In [4]:
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2
import tensorflow as tf

base_dir = os.path.join(root_dir, Config.PIPELINE_FOLDER)
file = [i for i in os.listdir(base_dir) if 'sqlite' in i]
config = os.path.join(base_dir, file[0])

connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = config

store = metadata_store.MetadataStore(connection_config)

In [7]:
previous_execution_status = get_latest_executions(store, Config.PIPELINE_NAME, 'Transform')

if previous_execution_status:
    previous_execution_status = previous_execution_status[0].last_known_state
else:
    raise Exception('[Exception] Run the Data Ingestion Notebook before Running this...') 
    
if  previous_execution_status == 3:
    print('[INFO] previous component Execution State is Success. You can Proceed Further now..')
elif previous_execution_status == 2:
    print('[Warning] previous component Execution is in Running State')

[INFO] previous component Execution State is Success. You can Proceed Further now..


## Defining the model

In this session, let assume we created some model which was trained on offline experimentation. we then decided to productionize it with the MLOps worflow. In this pipeline step, we want to export the model with our preprocessing steps, we need to guarantee that the model input names match the transformed feature names from ```preprocessing_fn()```. In our example model, we reuse the ```transformed_name()``` function to add the suffix _xf to our features.

In [4]:
%%writefile ../script/module.py

import tensorflow_transform as tft
import tensorflow as tf
import tensorflow_hub as hub
import sys
import os

root_dir = os.path.split(os.getcwd())[0]

sys.path.append(root_dir)
from utils.configurations.config import Config


def transformed_name(key):
    return key + '_xf'


def get_model():
    LABEL_KEY = "consumer_disputed"
    # Loop over the features and create an input for each feature.
    # Feature name, feature dimensionality.
    ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90
    }

    # Feature name, bucket count.
    BUCKET_FEATURES = {
    "zip_code": 10
    }

    # Feature name, value is unused.
    TEXT_FEATURES = {
    "consumer_complaint_narrative": None
    }
    # One-hot categorical features
    input_features = []
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
            name=transformed_name(key)))
        
    # Adding bucketized features
    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
            name=transformed_name(key)))
    
    # Adding text input features
    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,),
            name=transformed_name(key),
            dtype=tf.string))
    
    inputs = input_features + input_texts
    # Embed text features
    # Load the tf.hub module of the Universal Sentence Encoder model.
    MODULE_URL = Config.UNIVERSAL_EMBEDDING_MODEL
    embed = hub.KerasLayer(MODULE_URL)
    
    reshaped_narrative = tf.reshape(input_texts[0], [-1]) # Keras inputs are two-dimensional, 
                                                            # but the encoder expects one-dimensional inputs.
    embed_narrative = embed(reshaped_narrative)
    deep_ff = tf.keras.layers.Reshape((512, ), input_shape=(1, 512))(embed_narrative)
    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)
    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)
    both = tf.keras.layers.concatenate([deep, wide])
    output = tf.keras.layers.Dense(1, activation='sigmoid')(both)
    keras_model = tf.keras.models.Model(inputs, output)
    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='binary_crossentropy',
                    metrics=[
                        tf.keras.metrics.BinaryAccuracy(),
                        tf.keras.metrics.TruePositives()
                    ])
    return keras_model

Overwriting ../script/module.py


## TFX Trainer Component

The TFX Trainer component handles the training step in our pipeline. the Trainer component will produce a model that will be put into production.The Transform steps will be included in this model, the data preprocessing steps will always match what the model is expecting. This removes a huge potential source of errors when our model is deployed.

In our example project, the Trainer component requires the following inputs:
- The previously generated data schema, generated by the data validation step
- The transformed data and its preprocessing graph
- Training parameters (e.g., the number of training steps)
- A module file containing a run_fn() function, which defines the training process

### run_fn() Function

The Trainer component will look for a run_fn() function in our module file and use the function as an entry point to execute the training process. The module file needs The TFX Trainer Component to be accessible to the Trainer component. If we run the component in an interactive context, we can simply define the absolute path to the module file and pass it to the component. If you run your pipelines in production it will differ. It will be convered in the later notebook.

The run_fn() function is a generic entry point to the training steps and not tf.Keras specific. It carries out the following steps:
- Loading the training and validation data (or the data generator)
- Defining the model architecture and compiling the model
- Training the model
- Exporting the model to be evaluated in the next pipeline step

In [5]:
%%writefile ../script/module.py -a


def run_fn(fn_args):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output) # line 1
    train_dataset = input_fn(fn_args.train_files, tf_transform_output) # line 2
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output) # line 3
    model = get_model() # line 4
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch') # line 5
    model.fit( 
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=[tensorboard_callback])  # line 6
    
    signatures = {
    'serving_default':
    get_serve_tf_examples_fn(model,tf_transform_output).get_concrete_function(
            tf.TensorSpec(
            shape=[None],
            dtype=tf.string,
            name='examples')
            )
    } # line 7

    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures) # line 8
 


Appending to ../script/module.py


Line 1, 2, 3

The ```run_fn``` function receives a set of arguments, including the transform graph, example datasets, and training parameters through the fn_args object.

Loading data for model training and validation is performed in batches, and the loading is handled by the ```input_fn()``` function.

In [6]:
%%writefile ../script/module.py -a

LABEL_KEY = 'consumer_disputed'
def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
    tf_transform_output, batch_size=32):
    transformed_feature_spec = (
    tf_transform_output.transformed_feature_spec().copy())
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=transformed_name(LABEL_KEY))
    return dataset



Appending to ../script/module.py


The input_fn function lets us load the compressed, preprocessed datasets that were generated by the previous Transform step. To do this, we need to pass the tf_transform_output to the function. This gives us the data schema to load the dataset from the TFRecord data structures generated by the Transform component. By using the
preprocessed datasets, we can avoid data preprocessing during training and speed up the training process.


The ```input_fn``` returns a generator which created using the ```batched_features_dataset``` function that will supply data to the model one batch at a time.

**<u>line 4, 6</u>**

Now that we have defined our data-loading steps, the next step is defining our model architecture and compiling our model. In our run_fn , this will require a call to get_model() (line 4),


Next, we train our compiled tf.Keras model with the Keras method fit() (line 6)

Once the model training is complete, the next step is to export the trained model. we want to define how the preprocessing steps has to be exported with the model.

In line 7, we define the model signature and saved the model

The ```run_fn()``` exports the ```get_serve_tf_examples_fn``` as part of the model signature. When a model has been exported and deployed, every prediction request will pass through the ```serve_tf_examples_fn()```

With every request, we parse the serialized tf.Example records and apply the preprocessing steps to the raw
request data. The model then makes a prediction on the preprocessed data.

In [7]:
%%writefile ../script/module.py -a

def get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer() # Load the preprocessing graph

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        # Parse the raw tf.Example records from the request.
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        # Apply the preprocessing transformation to raw data.
        transformed_features = model.tft_layer(parsed_features)
        # Perform prediction with preprocessed data.
        outputs = model(transformed_features)
        return {'outputs': outputs}
    return serve_tf_examples_fn

Appending to ../script/module.py


## Running the Trainer Component

In [8]:
from tfx.types import channel_utils
from tfx.types import standard_artifacts
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2

In [9]:
artifacts = get_latest_artifacts(store, Config.PIPELINE_NAME, 'SchemaGen')
example_schema = find_latest_artifacts_by_type(store, artifacts, standard_artifacts.Schema.TYPE_NAME)
example_schema = channel_utils.as_channel(example_schema)

In [10]:
artifacts = get_latest_artifacts(store, Config.PIPELINE_NAME, 'Transform')
transform_examples = find_latest_artifacts_by_type(store, artifacts, standard_artifacts.Examples.TYPE_NAME)
transform_examples = channel_utils.as_channel(transform_examples)

transform_graph = find_latest_artifacts_by_type(store, artifacts, standard_artifacts.TransformGraph.TYPE_NAME)
transform_graph = channel_utils.as_channel(transform_graph)

In [11]:
TRAINING_STEPS = 1000
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file="../script/module.py",
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    transformed_examples=transform_examples,
    transform_graph=transform_graph,
    schema=example_schema,
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))

In [12]:
pipeline_name = Config.PIPELINE_NAME
base_root = os.path.split(os.getcwd())[0]
pipeline_root = os.path.join(base_root, f'temp_')
beam_args = [
    '--runner=DirectRunner'
]

if not os.path.exists(pipeline_root):
    raise Exception('Please do follow the notebook sequence')

context = InteractiveContext(pipeline_name = pipeline_name,
                            pipeline_root = pipeline_root,
                            beam_pipeline_args = beam_args)

context.run(trainer)

 108/1000 [==>...........................] - ETA: 4:28:54 - loss: 0.6927 - binary_accuracy: 0.5000 - true_positives: 3.000 - ETA: 15:15 - loss: 0.6957 - binary_accuracy: 0.4844 - true_positives: 3.5000  - ETA: 21:03 - loss: 0.6916 - binary_accuracy: 0.5104 - true_positives: 4.333 - ETA: 14:34 - loss: 0.6878 - binary_accuracy: 0.5352 - true_positives: 4.750 - ETA: 11:13 - loss: 0.6870 - binary_accuracy: 0.5494 - true_positives: 5.000 - ETA: 9:14 - loss: 0.6857 - binary_accuracy: 0.5628 - true_positives: 5.166 - ETA: 7:53 - loss: 0.6835 - binary_accuracy: 0.5762 - true_positives: 5.28 - ETA: 6:55 - loss: 0.6807 - binary_accuracy: 0.5896 - true_positives: 5.37 - ETA: 6:15 - loss: 0.6778 - binary_accuracy: 0.6013 - true_positives: 5.44 - ETA: 5:43 - loss: 0.6759 - binary_accuracy: 0.6096 - true_positives: 5.50 - ETA: 5:16 - loss: 0.6741 - binary_accuracy: 0.6164 - true_positives: 5.54 - ETA: 4:56 - loss: 0.6713 - binary_accuracy: 0.6238 - true_positives: 5.58 - ETA: 4:38 - loss: 0.6681 - b

 216/1000 [=====>........................] - ETA: 1:38 - loss: 0.5929 - binary_accuracy: 0.7323 - true_positives: 5.95 - ETA: 1:38 - loss: 0.5926 - binary_accuracy: 0.7325 - true_positives: 5.95 - ETA: 1:38 - loss: 0.5924 - binary_accuracy: 0.7327 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5922 - binary_accuracy: 0.7328 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5919 - binary_accuracy: 0.7330 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5917 - binary_accuracy: 0.7332 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5915 - binary_accuracy: 0.7334 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5912 - binary_accuracy: 0.7335 - true_positives: 5.95 - ETA: 1:37 - loss: 0.5910 - binary_accuracy: 0.7337 - true_positives: 5.95 - ETA: 1:36 - loss: 0.5908 - binary_accuracy: 0.7338 - true_positives: 5.95 - ETA: 1:36 - loss: 0.5906 - binary_accuracy: 0.7340 - true_positives: 5.95 - ETA: 1:36 - loss: 0.5903 - binary_accuracy: 0.7342 - true_positives: 5.95 - ETA: 1:36 - loss: 0.5901 - binary_accuracy:

 324/1000 [========>.....................] - ETA: 1:23 - loss: 0.5721 - binary_accuracy: 0.7462 - true_positives: 5.97 - ETA: 1:23 - loss: 0.5720 - binary_accuracy: 0.7462 - true_positives: 5.97 - ETA: 1:23 - loss: 0.5719 - binary_accuracy: 0.7463 - true_positives: 5.97 - ETA: 1:23 - loss: 0.5718 - binary_accuracy: 0.7464 - true_positives: 5.97 - ETA: 1:23 - loss: 0.5716 - binary_accuracy: 0.7465 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5715 - binary_accuracy: 0.7466 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5714 - binary_accuracy: 0.7466 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5713 - binary_accuracy: 0.7467 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5711 - binary_accuracy: 0.7468 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5710 - binary_accuracy: 0.7468 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5709 - binary_accuracy: 0.7469 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5708 - binary_accuracy: 0.7470 - true_positives: 5.97 - ETA: 1:22 - loss: 0.5707 - binary_accuracy:

 432/1000 [===========>..................] - ETA: 1:09 - loss: 0.5615 - binary_accuracy: 0.7524 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5614 - binary_accuracy: 0.7524 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5613 - binary_accuracy: 0.7524 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5613 - binary_accuracy: 0.7525 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5612 - binary_accuracy: 0.7525 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5611 - binary_accuracy: 0.7526 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5610 - binary_accuracy: 0.7526 - true_positives: 5.98 - ETA: 1:09 - loss: 0.5610 - binary_accuracy: 0.7527 - true_positives: 5.98 - ETA: 1:08 - loss: 0.5609 - binary_accuracy: 0.7527 - true_positives: 5.98 - ETA: 1:08 - loss: 0.5608 - binary_accuracy: 0.7527 - true_positives: 5.98 - ETA: 1:08 - loss: 0.5607 - binary_accuracy: 0.7528 - true_positives: 5.98 - ETA: 1:08 - loss: 0.5607 - binary_accuracy: 0.7528 - true_positives: 5.98 - ETA: 1:08 - loss: 0.5606 - binary_accuracy:

 540/1000 [===============>..............] - ETA: 57s - loss: 0.5548 - binary_accuracy: 0.7558 - true_positives: 5.988 - ETA: 57s - loss: 0.5548 - binary_accuracy: 0.7558 - true_positives: 5.988 - ETA: 57s - loss: 0.5547 - binary_accuracy: 0.7558 - true_positives: 5.988 - ETA: 57s - loss: 0.5547 - binary_accuracy: 0.7559 - true_positives: 5.988 - ETA: 57s - loss: 0.5546 - binary_accuracy: 0.7559 - true_positives: 5.988 - ETA: 57s - loss: 0.5546 - binary_accuracy: 0.7559 - true_positives: 5.988 - ETA: 56s - loss: 0.5545 - binary_accuracy: 0.7559 - true_positives: 5.988 - ETA: 56s - loss: 0.5545 - binary_accuracy: 0.7560 - true_positives: 5.988 - ETA: 56s - loss: 0.5544 - binary_accuracy: 0.7560 - true_positives: 5.988 - ETA: 56s - loss: 0.5544 - binary_accuracy: 0.7560 - true_positives: 5.988 - ETA: 56s - loss: 0.5543 - binary_accuracy: 0.7560 - true_positives: 5.988 - ETA: 56s - loss: 0.5543 - binary_accuracy: 0.7560 - true_positives: 5.988 - ETA: 56s - loss: 0.5542 - binary_accuracy: 

...

1000/1000 [==============================] - ETA: 2s - loss: 0.5414 - binary_accuracy: 0.7602 - true_positives: 11.326 - ETA: 2s - loss: 0.5414 - binary_accuracy: 0.7602 - true_positives: 11.338 - ETA: 2s - loss: 0.5414 - binary_accuracy: 0.7602 - true_positives: 11.350 - ETA: 2s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.362 - ETA: 2s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.374 - ETA: 2s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.386 - ETA: 2s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.398 - ETA: 2s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.410 - ETA: 1s - loss: 0.5413 - binary_accuracy: 0.7602 - true_positives: 11.422 - ETA: 1s - loss: 0.5412 - binary_accuracy: 0.7602 - true_positives: 11.433 - ETA: 1s - loss: 0.5412 - binary_accuracy: 0.7602 - true_positives: 11.445 - ETA: 1s - loss: 0.5412 - binary_accuracy: 0.7602 - true_positives: 11.457 - ETA: 1s - loss: 0.5412 - binary_accuracy: 0

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jagan-ds/Documents/Tensorflow-Extended-tutorial/temp_/Trainer/model/22/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jagan-ds/Documents/Tensorflow-Extended-tutorial/temp_/Trainer/model/22/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 22
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 35
        type_id: 17
        uri: "/home/jagan-ds/Documents/Tensorflow-Extended-tutorial/temp_/Trainer/model/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 36
        type_id: 18
        uri: "/home/jagan-ds/Documents/Tensorflow-Extended-tutorial/temp_/Trainer/model_run/22"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

After the model training and exporting is completed, the component will register the path of the exported model with the metadata store. Downstream components can pick up the model for the model validation.
The Trainer component is generic and not limited to running TensorFlow models. However, the components later in the pipeline expect that the model is saved in the TensorFlow SavedModel format. The SavedModel graph includes the Transform graph, so the data preprocessing steps are part of the model now.

>**note:**<br>
>> ***Overriding the Trainer Component’s Executor***<br>
we override the Trainer component’s executor to enable the generic training entry point ```run_fn()``` function instead of the default ```trainer_fn()``` function, which only supports ```tf.Estimator``` models. In later notebook, we will see another Trainer executor, the ```ai_platform_trainer_executor.GenericExecutor```. This executor allows you to train models on Google Cloud’s AI Platform instead of inside your pipeline. This is an alternative if your model requires specific training hardware (e.g., GPUs or tensor processing units [TPUs]), which aren’t available in your pipeline environment.

To view TensorBoard in a notebook, we get the location of the model training logs and pass it to TensorBoard

In [14]:
model_dir = trainer.outputs['model'].get()[0].uri

%tensorboard --logdir {model_dir}

### Using the SavedModel outside a pipeline

If we would like to inspect the exported SavedModel outside a TFX pipeline, we can load the model as a concrete function, which represents the graph of a single signature

In [17]:
model_path = trainer.outputs.model.get()[0].uri
model_path = os.path.join(model_path, 'serving_model_dir')
model = tf.saved_model.load(export_dir=model_path)
predict_fn = model.signatures["serving_default"]

With the model loaded as a concrete function, we can now perform predictions. The exported model expects the input data to be provided in the tf.Example data structure

In [36]:
example = tf.train.Example(features=tf.train.Features(feature={
                    "product": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Debt collection'.encode()])),
                    "sub_product": tf.train.Feature(bytes_list=tf.train.BytesList(value=['I do not know'.encode()])),
                    "issue": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Disclosure verification of debt'.encode()])),
                    "sub_issue": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Right to dispute notice not received'.encode()])),
                    "state": tf.train.Feature(bytes_list=tf.train.BytesList(value=['NY'.encode()])),
                    "zip_code": tf.train.Feature(bytes_list=tf.train.BytesList(value=['113XX'.encode()])),
                    "company": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Encore Capital Group'.encode()])),
                    "company_response": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Closed with explanation'.encode()])),
                    "timely_response": tf.train.Feature(bytes_list=tf.train.BytesList(value=['Yes'.encode()])),
                    "consumer_complaint_narrative": tf.train.Feature(bytes_list=tf.train.BytesList(value=['I was denied employment because of a judgment against me.  I was/ am completely unaware of any hearing, never received any notice of collection of a debt by Midland LLC .  Midland LLC apparently took me to court somewhere without serving me any documents, and won a courts judgement.  I was never notified of any hearing, complaint, or received notice of collection of a debt by Midland LLC.'.encode()]))
}))

serialized_example = example.SerializeToString()
prediction = predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0,0]
print(f'predicted probability : {prediction}')
print(f'predicted value : {int(prediction)}')

predicted probability : 0.3753308951854706
predicted value : 0


## Distribution Strategies

We done with the trainer component what if our model needs more computation power to be utilized to proceed training faseter?

Therefore, TensorFlow provides distribution strategies for machine learning models that can’t be adequately trained on a single GPU. You might want to consider distribution strategies when you want to accelerate your training or you can’t fit the entire model into a single GPU. The strategies we describe here are abstractions to distribute the model parameters across multiple GPUs or even multiple servers. In general, there are two groups of
strategies: 
- synchronous
- asynchronous

Under the synchronous strategies, all training workers train with different slices of the training data synchronously and then aggregate the gradients from all workers before updating the model. 

The asynchronous strategies train models independently with the entire dataset on differentworkers. Each worker updates the gradients of the model asynchronously, without waiting for the other workers to finish. 

Typically, synchronous strategies are coordinated  via all-reduce operations and asynchronous strategies through a parameter server architecture. A few synchronous and asynchronous strategies exist, and they have their benefits
and drawbacks.

Types of statergies in tensorflow are:
- MirroredStrategy
- TPUStrategy
- MultiWorkerMirroredStrategy
- CentralStorageStrategy
- ParameterServerStrategy

### MirroredStrategy
```tf.distribute.MirroredStrategy``` supports synchronous distributed training on multiple GPUs on one machine. It creates one replica per GPU device. Each variable in the model is mirrored across all the replicas. Together, these variables form a single conceptual variable called MirroredVariable. These variables are kept in sync with each other by applying identical updates.

Efficient all-reduce algorithms are used to communicate the variable updates across the devices. All-reduce aggregates tensors across all the devices by adding them up, and makes them available on each device.


### TPUStrategy
```tf.distribute.TPUStrategy``` lets you run your TensorFlow training on Tensor Processing Units (TPUs). TPUs are Google's specialized ASICs designed to dramatically accelerate machine learning workloads. They are available on Google Colab, the TensorFlow Research Cloud and Cloud TPU.

In terms of distributed training architecture, TPUStrategy is the same ```MirroredStrategy```, it implements synchronous distributed training. TPUs provide their own implementation of efficient all-reduce and other collective operations across multiple TPU cores, which are used in TPUStrategy.


### MultiWorkerMirroredStrategy
```tf.distribute.MultiWorkerMirroredStrategy``` is very similar to MirroredStrategy. It implements synchronous distributed training across multiple workers, each with potentially multiple GPUs. Similar to tf.distribute.MirroredStrategy, it creates copies of all variables in the model on each device across all workers.


### ParameterServerStrategy
Parameter server training is a common data-parallel method to scale up model training on multiple machines. A parameter server training cluster consists of workers and parameter servers. Variables are created on parameter servers and they are read and updated by workers in each step.


### CentralStorageStrategy
```tf.distribute.experimental.CentralStorageStrategy``` does synchronous training as well. Variables are not mirrored, instead they are placed on the CPU and operations are replicated across all local GPUs. If there is only one GPU, all variables and operations will be placed on that GPU.

![distribution_stratergy](image/distribution_stratergy.png)

An example of it here. We can apply the MirroredStrategy easily by adding a few lines before
invoking our model creation and the subsequent model.compile() call.

for that replace the ```model=get_model()``` (line 4) in ```run_n()``` with the code below 
```
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
model = get_model()
```

This will help us the train our model in a distributed setup. we wrap the model creation and compilation with the Python manager (in our case, it all happens inside of the get_model() function). This will create and compile our model under the distribution scope of our choice. The ```MirroredStrategy``` will use all available GPUs of the instance. If you want to reduce the number of GPU instances being used, you can specify the GPUs to be used with the MirroredStrategy by changing the creation of the distribution strategy:

```
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"]
```

In this example, we specify two GPUs to be used for our training runs. These distribution strategies are useful for large training jobs that won’t fit on the memory of a single GPU.

> note: <br>
Batch Size Requirement When Using the MirroredStrategy.The MirroredStrategy expects that the batch size is proportional to the number of devices. For example, if you train with five GPUs, the batch size needs to be a multiple of the number of GPUs. Please keep this in mind when you set up your input_fn()

## Hyperparameter Tuning in TFX Pipelines
In a TFX pipeline, hyperparameter tuning takes in the data from the Transform com‐
ponent and trains a variety of models to establish the best hyperparameters. The
hyperparameters are then passed to the Trainer component, which then trains a final
model using them.
In this case, the model definition function (the get_model function in our example)
needs to accept the hyperparameters as an input and build the model according to
the specified hyperparameters. So, for example, the number of layers needs to be
defined as an input argument.